<a href="https://colab.research.google.com/github/egitonga395/kamilimu_assignments/blob/sql/Ericksqlassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Load datasets
athletes = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/athlete_events.csv")
regions = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/noc_regions.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')


# Connect to SQLite in-memory DB
conn = sqlite3.connect(":memory:")

# Write DataFrames to SQL tables
athletes.to_sql("athletes_table", conn, index=False, if_exists="replace")
regions.to_sql("regions_table", conn, index=False, if_exists="replace")
sales.to_sql("sales_table", conn, index=False, if_exists="replace")


4622

show the how te data looks like first
s

In [ ]:
pd.read_sql("SELECT * FROM athletes_table LIMIT 5", conn)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


show the first five country on basis of having gold

In [ ]:
pd.read_sql("""
SELECT noc, COUNT(*) AS gold_count
FROM athletes_table
WHERE medal = 'Gold'
GROUP BY noc
ORDER BY gold_count DESC
LIMIT 5
""", conn)

,NOC,gold_count
0,USA,2638
1,URS,1082
2,GER,745
3,GBR,678
4,ITA,575


How many distinct events are there in each sport

In [ ]:
pd.read_sql("""
SELECT sport, COUNT(DISTINCT event) AS distinct_events
FROM athletes_table
GROUP BY sport
ORDER BY COUNT(DISTINCT event) DESC
""", conn)

,Sport,distinct_events
0,Shooting,83
1,Athletics,83
2,Swimming,55
3,Cycling,44
4,Sailing,38
...,...,...
61,Cricket,1
62,Basque Pelota,1
63,Baseball,1
64,Alpinism,1


Show all athletes from the United States (NOC = 'USA').

In [ ]:
pd.read_sql("""
SELECT name
FROM athletes_table
WHERE noc = 'USA'
""", conn)

,Name
0,Per Knut Aaland
1,Per Knut Aaland
2,Per Knut Aaland
3,Per Knut Aaland
4,Per Knut Aaland
...,...
18848,Rami Zur
18849,Rami Zur
18850,Rami Zur
18851,"Victor Andrew ""Vic"" Zwolak"


Count of medals awarded each year.

In [ ]:
pd.read_sql("""
SELECT year, COUNT(medal) AS medal_count
FROM athletes_table
WHERE medal IS NOT NULL
GROUP BY year
ORDER BY year
""", conn)

,Year,medal_count
0,1896,143
1,1900,604
2,1904,486
3,1906,458
4,1908,831
5,1912,941
6,1920,1308
7,1924,962
8,1928,823
9,1932,739


athlete's records where height or weight is missing

In [ ]:
pd.read_sql("""
SELECT name AS athlete_name, height, weight
FROM athletes_table
WHERE height IS NULL AND weight IS NULL
""", conn)

,athlete_name,Height,Weight
0,Gunnar Nielsen Aaby,None,None
1,Edgar Lindenau Aabye,None,None
2,"Einar Ferdinand ""Einari"" Aalto",None,None
3,Arvo Ossian Aaltonen,None,None
4,Arvo Ossian Aaltonen,None,None
...,...,...,...
58778,Marius Edmund Zwiller,None,None
58779,Werner Zwingli,None,None
58780,Werner Zwingli,None,None
58781,Jan (Johann-) Zybert (Siebert-),None,None


replacing the null value with average athelete weight or height

In [ ]:
pd.read_sql("""
SELECT name AS athlete_name,
height, weight,
       CASE
           WHEN height IS NULL THEN (SELECT AVG(height) FROM athletes_table)
           ELSE height
       END AS height,
       CASE
           WHEN weight IS NULL THEN (SELECT AVG(weight) FROM athletes_table)
           ELSE weight
       END AS weight
FROM athletes_table
""", conn)

,athlete_name,Height,Weight,height,weight
0,A Dijiang,180.0,80.0,180.00000,80.000000
1,A Lamusi,170.0,60.0,170.00000,60.000000
2,Gunnar Nielsen Aaby,NaN,NaN,175.33897,70.702393
3,Edgar Lindenau Aabye,NaN,NaN,175.33897,70.702393
4,Christine Jacoba Aaftink,185.0,82.0,185.00000,82.000000
...,...,...,...,...,...
271111,Andrzej ya,179.0,89.0,179.00000,89.000000
271112,Piotr ya,176.0,59.0,176.00000,59.000000
271113,Piotr ya,176.0,59.0,176.00000,59.000000
271114,Tomasz Ireneusz ya,185.0,96.0,185.00000,96.000000


look at how the table looks like first

In [ ]:
pd.read_sql("SELECT * FROM sales_table LIMIT 5", conn)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,None,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


return total sale per item

In [ ]:
pd.read_sql("""
SELECT CAST(26.5 AS int) INT_price
""", conn)

,INT_price
0,26


In [ ]:
pd.read_sql("""
PRAGMA table_info(sales_table);
""", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,0
1,1,quantity,INTEGER,0,None,0
2,2,item_name,TEXT,0,None,0
3,3,choice_description,TEXT,0,None,0
4,4,item_price,TEXT,0,None,0


In [ ]:
pd.read_sql("""
SELECT CAST(item_price AS int) INT_price
FROM sales_table
""", conn)


,INT_price
0,0
1,0
2,0
3,0
4,0
...,...
4617,0
4618,0
4619,0
4620,0


In [ ]:
pd.read_sql("""
SELECT order_id, quantity, item_name, choice_description, item_price, CAST(REPLACE(item_price,'$','') AS float)* quantity AS total_sale
FROM sales_table
""", conn)

,order_id,quantity,item_name,choice_description,item_price,total_sale
0,1,1,Chips and Fresh Tomato Salsa,None,$2.39,2.39
1,1,1,Izze,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,33.96
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


top items with highst price

In [ ]:
pd.read_sql("""
SELECT item_name, CAST(REPLACE(item_price,'$','') AS float) price
FROM sales_table
ORDER BY price DESC
LIMIT 5
""", conn)

,item_name,price
0,Chips and Fresh Tomato Salsa,44.25
1,Carnitas Bowl,35.25
2,Chicken Burrito,35.00
3,Chicken Burrito,35.00
4,Veggie Burrito,33.75


count distict customers from order ids

In [ ]:
pd.read_sql("""
SELECT COUNT(DISTINCT order_id) AS distinct_customers
FROM sales_table
""", conn)

,distinct_customers
0,1834


Section 2
count the number of atheletes with a medal(is not null) for each country
then find their average age and classify


# **Thought process**


1. group by coutry
2. then count them
3. find the average age of medalist(with medal) is not none
4. classify them based on age

*    
*   grouby them by country
*   count ANY with one or more medals
*   find their age
*   then classify














In [ ]:
pd.read_sql("SELECT * FROM athletes_table LIMIT 5", conn)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


In [ ]:
pd.read_sql("""
SELECT NOC, COUNT(DISTINCT Name) AS total_medalists, AVG(Age) AS average_age,
CASE
  WHEN AVG(Age) < 25 THEN 'High'
  WHEN AVG(Age) >= 25 AND AVG(Age) <=30 THEN 'Medium'
  WHEN AVG(Age) > 25 THEN 'Low'
END AS perfomance
FROM athletes_table noc
WHERE medal IS NOT NULL
GROUP BY NOC
ORDER BY COUNT(DISTINCT Name) DESC""", conn)




,NOC,total_medalists,average_age,perfomance
0,AFG,1,23.000000,High
1,AHO,1,19.000000,High
2,BAR,1,24.000000,High
3,BER,1,25.000000,Medium
4,BOT,1,18.000000,High
...,...,...,...,...
144,FRA,1277,27.982153,Medium
145,GER,1518,26.416510,Medium
146,GBR,1601,27.846310,Medium
147,URS,1675,25.268877,Medium
